In [1]:
import os, re
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
from transformers import RobertaTokenizer, RobertaConfig, TFRobertaForSequenceClassification
from tokenizers import ByteLevelBPETokenizer
pd.options.display.max_columns = 30

# Get the Data

In [2]:
x_train, y_train = fetch_20newsgroups(subset='train', return_X_y=True)
x_valid, y_valid = fetch_20newsgroups(subset='test', return_X_y=True)

# Build Tokenizer

In [3]:
os.mkdir('text_files')
for e, text in enumerate(x_train):
    with open(f"text_files/train_{e+1:05}.txt", 'w') as f:
        f.write(re.sub(r'\s+', ' ', text))

In [4]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(
    files=[f"text_files/train_{e+1:05}.txt" for e in range(len(x_train))], vocab_size=30_522,
    special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>']
)
os.mkdir('tokenizer')
tokenizer.save_model('tokenizer')

['tokenizer/vocab.json', 'tokenizer/merges.txt']

# Tokenize Data

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('tokenizer')
def tokenize(texts):
    return tokenizer(
        texts, padding='max_length', truncation=True, max_length=512, return_tensors='np'
    )

In [6]:
x_train_tokenized = tokenize(x_train)
x_valid_tokenized = tokenize(x_valid)

# Prepare Data Loaders

In [7]:
train_data = tf.data.Dataset.from_tensor_slices((dict(x_train_tokenized), y_train)).batch(8)
valid_data = tf.data.Dataset.from_tensor_slices((dict(x_valid_tokenized), y_valid)).batch(8)

2022-07-02 09:02:02.597933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 09:02:02.598987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 09:02:02.599670: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-02 09:02:02.602330: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

# Build Model

In [8]:
config = RobertaConfig(
    vocab_size=30_522,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1, 
    num_labels=20,
)
model = TFRobertaForSequenceClassification(config)

# Train

In [9]:
model.compile(
    optimizer=tf.optimizers.Adam(1e-4), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics='accuracy'
)
model.fit(train_data, validation_data=valid_data, epochs=1)
model.save_pretrained('news-classifier')

2022-07-02 09:02:19.420427: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1415/1415 [==============================] - 517s 356ms/step - loss: 3.0515 - accuracy: 0.0495 - val_loss: 3.0625 - val_accuracy: 0.0530


# Inference

In [10]:
model = TFRobertaForSequenceClassification.from_pretrained('news-classifier')
logits = model.predict(valid_data, verbose=1).logits
preds_proba = tf.nn.softmax(logits).numpy()
preds = preds_proba.argmax(axis=1)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at news-classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


942/942 [==============================] - 92s 96ms/step


# Evaluation

In [11]:
clf_report = pd.DataFrame(confusion_matrix(y_valid, preds))
precision, recall, fscore, support = precision_recall_fscore_support(y_valid, preds)
clf_report['precision'] = precision
clf_report['recall'] = recall
clf_report['fscore'] = fscore
clf_report['support'] = support
clf_report

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,precision,recall,fscore,support
0,0,0,0,0,0,0,0,0,0,0,319,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,319
1,0,0,0,0,0,0,0,0,0,0,389,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,389
2,0,0,0,0,0,0,0,0,0,0,394,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,394
3,0,0,0,0,0,0,0,0,0,0,392,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,392
4,0,0,0,0,0,0,0,0,0,0,385,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,385
5,0,0,0,0,0,0,0,0,0,0,395,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,395
6,0,0,0,0,0,0,0,0,0,0,390,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,390
7,0,0,0,0,0,0,0,0,0,0,396,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,396
8,0,0,0,0,0,0,0,0,0,0,398,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,398
9,0,0,0,0,0,0,0,0,0,0,397,0,0,0,0,0,0,0,0,0,0.000000,0.0,0.000000,397
